In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from nltk.translate.bleu_score import sentence_bleu
from tqdm import tqdm
from pathlib import Path
pd.set_option('max_colwidth', None)
pd.set_option('max_columns', None)

In [3]:
PATH_ROOT = Path.cwd().parents[1].resolve()
%cd {PATH_ROOT}

/Users/Adam/PycharmProjects/SOTA


In [4]:
df = pd.read_csv(os.path.join(PATH_ROOT,"data/combined/with_annotators/combined_dataset.csv"), index_col=0)

In [5]:
df.head(3)

,annotator,text1,text2,label,dataset,random,duration,total_seconds,pair_id,reduced_label,...,bleu1,glove_cosine,fasttext_cosine,BertScore,chrfScore,POS Dist score,1-gram_overlap,ROUGE-1,ROUGE-2,ROUGE-l
0,A3BCGN11HDM8QR,"And he sent Eliakim , who was over the househo...","And he sent Eliakim , who was over the house ,...",2,bible_human,0,0 days 00:00:12.000000000,12,pair_0,-1,...,0.777466,0.015252,0.035903,0.969967,0.699194,1.232655,0.612903,0.782609,0.626866,0.75
1,A3SQ00HYQN7FYB,"And he sent Eliakim , who was over the househo...","And he sent Eliakim , who was over the house ,...",3,bible_human,0,0 days 00:00:12.000000000,12,pair_0,0,...,0.777466,0.015252,0.035903,0.969967,0.699194,1.232655,0.612903,0.782609,0.626866,0.75
2,A5WAWW70PYRP,"And he sent Eliakim , who was over the househo...","And he sent Eliakim , who was over the house ,...",4,bible_human,0,0 days 00:07:19.000000000,439,pair_0,1,...,0.777466,0.015252,0.035903,0.969967,0.699194,1.232655,0.612903,0.782609,0.626866,0.75


In [24]:
df = pd.read_csv(PATH_ROOT / 'data' / 'raw_data' / 'with_annotators' / 'Bible_human.csv', index_col=0)

In [25]:
print(df.columns.tolist())
print(df.shape)

['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'RequesterAnnotation', 'AssignmentDurationInSeconds', 'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime', 'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime', 'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate', 'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.text1', 'Input.text2', 'Answer.semantic-similarity.label', 'Approve', 'Reject']
(3000, 25)


In [26]:
df.groupby(['Input.text1', 'Input.text1']).size().mean()

3.0120481927710845

In [27]:
df.head()

,HITId,HITTypeId,Title,Description,Keywords,RequesterAnnotation,AssignmentDurationInSeconds,AssignmentId,WorkerId,AssignmentStatus,AcceptTime,SubmitTime,AutoApprovalTime,ApprovalTime,RejectionTime,RequesterFeedback,WorkTimeInSeconds,LifetimeApprovalRate,Last30DaysApprovalRate,Last7DaysApprovalRate,Input.text1,Input.text2,Answer.semantic-similarity.label,Approve,Reject
0,3T5ZXGO9DEODEIR3CPS5EBLJMKVQZX,3TZJMY6N8MTM6A01KQ0QESH9DVHG7U,How similar is the meaning of these two pieces of text?,How similar is the meaning of these two pieces of text?,"meaning, semantic, similarity, text",BatchId:3851006;OriginalHitTemplateId:928390905;,14400,38SKSKU7R1XIEEK9AI1OMDFLVZJILV,A3BCGN11HDM8QR,Approved,Wed Nov 27 08:39:22 PST 2019,Wed Nov 27 08:39:34 PST 2019,Sat Nov 30 08:39:34 PST 2019,Wed Nov 27 11:05:22 PST 2019,NaN,NaN,12,100% (142/142),100% (142/142),100% (142/142),"And he sent Eliakim , who was over the household , and Shebna the scribe , and the elders of the priests , covered with sackcloth , unto Isaiah the prophet the son of Amoz .","And he sent Eliakim , who was over the house , and Shebna the scribe , and the chief priests , dressed in haircloth , to Isaiah the prophet , the son of Amoz .",2,NaN,NaN
1,3T5ZXGO9DEODEIR3CPS5EBLJMKVQZX,3TZJMY6N8MTM6A01KQ0QESH9DVHG7U,How similar is the meaning of these two pieces of text?,How similar is the meaning of these two pieces of text?,"meaning, semantic, similarity, text",BatchId:3851006;OriginalHitTemplateId:928390905;,14400,3SPJ0334213Y9OBEBFBESMWHHTBJYZ,A3SQ00HYQN7FYB,Approved,Wed Nov 27 08:38:14 PST 2019,Wed Nov 27 08:38:26 PST 2019,Sat Nov 30 08:38:26 PST 2019,Wed Nov 27 11:05:23 PST 2019,NaN,NaN,12,100% (123/123),100% (123/123),100% (123/123),"And he sent Eliakim , who was over the household , and Shebna the scribe , and the elders of the priests , covered with sackcloth , unto Isaiah the prophet the son of Amoz .","And he sent Eliakim , who was over the house , and Shebna the scribe , and the chief priests , dressed in haircloth , to Isaiah the prophet , the son of Amoz .",3,NaN,NaN
2,3T5ZXGO9DEODEIR3CPS5EBLJMKVQZX,3TZJMY6N8MTM6A01KQ0QESH9DVHG7U,How similar is the meaning of these two pieces of text?,How similar is the meaning of these two pieces of text?,"meaning, semantic, similarity, text",BatchId:3851006;OriginalHitTemplateId:928390905;,14400,3ZOTGHDK5IBFEDQ2SMBUIWP2Z2PSOD,A5WAWW70PYRP,Approved,Wed Nov 27 08:38:38 PST 2019,Wed Nov 27 08:45:57 PST 2019,Sat Nov 30 08:45:57 PST 2019,Wed Nov 27 11:05:23 PST 2019,NaN,NaN,439,100% (38/38),100% (38/38),100% (38/38),"And he sent Eliakim , who was over the household , and Shebna the scribe , and the elders of the priests , covered with sackcloth , unto Isaiah the prophet the son of Amoz .","And he sent Eliakim , who was over the house , and Shebna the scribe , and the chief priests , dressed in haircloth , to Isaiah the prophet , the son of Amoz .",4,NaN,NaN
3,3VDI8GSXAFT7HXW203NG7DKDHOG8GG,3TZJMY6N8MTM6A01KQ0QESH9DVHG7U,How similar is the meaning of these two pieces of text?,How similar is the meaning of these two pieces of text?,"meaning, semantic, similarity, text",BatchId:3851006;OriginalHitTemplateId:928390905;,14400,3COPXFW7XBCZBUPV2DF7GA3ISD7KP4,AE608VRXK6M2E,Approved,Wed Nov 27 08:35:29 PST 2019,Wed Nov 27 08:45:41 PST 2019,Sat Nov 30 08:45:41 PST 2019,Wed Nov 27 11:06:19 PST 2019,NaN,NaN,612,100% (38/38),100% (38/38),100% (38/38),"And Ibneiah , the son of Jeroham , and Elah , the son of Uzzi , the son of Michri , and Meshullam , the son of Shephatiah , the son of Reuel , the son of Ibnijah ;","and Ibneiah son of Jeroham , and Elah son of Uzzi , son of Michri , and Meshullam son of Shephatiah , son of Reuel , son of Ibnijah .",5 - Highly Similar,NaN,NaN
4,3VDI8GSXAFT7HXW203NG7DKDHOG8GG,3TZJMY6N8MTM6A01KQ0QESH9DVHG7U,How similar is the meaning of these two pieces of text?,How similar is the meaning of these two pieces of text?,"meaning, semantic, similarity, text",BatchId:3851006;OriginalHitTemplateId:928390905;,14400,3LS2AMNW5FQ31XBP

In [ ]:
# # Removing Outliers
# 
# Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling, we want to currently:
# 
# 1. Find out who the bad actors are
# 2. See if there is any overlap from the dif. category of bad actors
# 
# (Removing bad phrases should also be done - but is beyond the scope of this notebook)
# 
# 
# The categories of bad actors are:
# 
# [x] - Slow Annotators - ba_time </br>
# [x] - Greater variance in random datasets than non-random datasets - ba_unvar_annotations </br>
# [x] - Unpopular (disagree with two others often) - ba_unpopular </br>
# [ ] - Inconsistent with sentiment != semantics - ba_semantics </br>

# # Removing Outliers
# 
# Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling,we want to currently:
 
 1. Find out who the bad actors are
 2. See if there is any overlap from the dif. category of bad actors
 
 (Removing bad phrases should also be done - but is beyond the scope of this notebook)
 
 
 The categories of bad actors are:
 
 [x] - Slow Annotators </br>
 [x] - Greater variance in random datasets than non-random datasets </br>
[ ] - Unpopular (disagree with two others often) </br>
 [ ] - Inconsistent with sentiment != semantics </br>

# Removing Outliers
 
Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling,we want to currently:
 
 1. Find out who the bad actors are
 2. See if there is any overlap from the dif. category of bad actors
 
 (Removing bad phrases should also be done - but is beyond the scope of this notebook)
 
 
 The categories of bad actors are:
 
 [x] - Slow Annotators </br>
 [x] - Greater variance in random datasets than non-random datasets </br>
 [ ] - Unpopular (disagree with two others often) </br>
 [ ] - Inconsistent with sentiment != semantics

# Removing Outliers
 
Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling,we want to currently:
 
 1. Find out who the bad actors are
 2. See if there is any overlap from the dif. category of bad actors
 
 (Removing bad phrases should also be done - but is beyond the scope of this notebook)
 
 
 The categories of bad actors are:
 
 [x] - Slow Annotators </br>
 [x] - Greater variance in random datasets than non-random datasets </br>
 [ ] - Unpopular (disagree with two others often) </br>
 [ ] - Inconsistent with sentiment != semantics </br>

# Removing Outliers

Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling, we want to currently:

1. Find out who the bad actors are
2. See if there is any overlap from the dif. category of bad actors

(Removing bad phrases should also be done - but is beyond the scope of this notebook)


The categories of bad actors are:

[x] - Slow Annotators </br>
[x] - Greater variance in random datasets than non-random datasets </br>
[ ] - Unpopular (disagree with two others often) </br>
[ ] - Inconsistent with sentiment != semantics </br>

## Time Outliers

Under the assumption that anyone that takes over the 95 percentile of time.

In [ ]:
print(df.total_seconds.describe(percentiles = [.25,.5,.75,.9,.95]))

# ba = bad actor
df['mean_annotation_time'] = df.groupby('annotator').total_seconds.transform('mean')
print(df.mean_annotation_time.describe(percentiles = [.25,.5,.75,.9,.95]))
ba_time = df[df.mean_annotation_time > 405].annotator.unique().tolist()
print(len(df[df.total_seconds > 336].annotator.unique().tolist()))
print(len(ba_time))

## Unvarianced Annotations
Labelers whos std is too low mean non-random - random difference is too high  

In [ ]:
labelers = df[df.random==0].groupby(['annotator'])['label'].agg(['size','mean','std','min','max'])
labelers = labelers[labelers['size']>1]
#df = df[df.annotator.apply(lambda x:x in set(labelers.index))]

labelers_rand = df[df.random==1].groupby(['annotator'])['label'].agg(['size','mean','std','min','max'])
labelers_rand = labelers_rand[labelers_rand['size']>1]
labelers = labelers.join(labelers_rand, rsuffix = '_rand')
labelers['mean_random_gap'] = labelers['mean']-labelers['mean_rand']
labelers['std_ratio'] = labelers['std']/labelers['std_rand']

total_std = df.groupby('annotator')['label'].std()
total_std.name = 'total_std'
labelers = labelers.join(total_std)

In [ ]:
ba_unvar_annotations = labelers[(labelers.total_std<1) & (labelers.mean_random_gap < 0)].index.tolist()
len(ba_unvar_annotations)

## Unpopular Annotators
Those who over 50% of the time, disagree with the other annotators (in the reduced label)

In [ ]:
df_uniquelabels = df.groupby("pair_id")["reduced_label"].nunique()
pairs_twoagree = df_uniquelabels[(df.groupby("pair_id")["reduced_label"].nunique() == 2).values].index.tolist()
df_twoagree = df[df["pair_id"].isin(pairs_twoagree)]

df_twoagree['generally_accepted_label'] = df_twoagree.groupby("pair_id")['reduced_label'].transform('median')

In [ ]:
df_twoagree.head()

In [ ]:
df_unpopularopinion = df_twoagree[df_twoagree.reduced_label != df_twoagree.generally_accepted_label].groupby('annotator').size().reset_index()
df_unpopularopinion.columns = ['annotator','unpopular_opinion']

df_allopinions = df[df['annotator'].isin(list(df_unpopularopinion.annotator))].groupby('annotator').size().reset_index()
df_allopinions.columns = ['annotator','all_opinion']

df_opinion_all_unpop = df_allopinions.merge(df_unpopularopinion,on="annotator")

In [ ]:
df_opinion_all_unpop.head()

In [ ]:
ba_unpopular = df_opinion_all_unpop[((df_opinion_all_unpop.unpopular_opinion / df_opinion_all_unpop.all_opinion) > 0.5) & (df_opinion_all_unpop.all_opinion > 4)].annotator.tolist()

## Sentiment // Semantic Understanding

In [ ]:
# Sentiment analysis pipeline
sentiment_pipe = pipeline("sentiment-analysis")

In [ ]:
from tqdm import tqdm

text1_sent,text2_sent =[], []

pbar = tqdm(total = len(df)//100+1, position = 0, leave = True)
for i in range (len(df)//100+1):
    t1_s = sentiment_pipe(df.text1.tolist()[100*i:np.min([100*i+100,len(df)])])
    t2_s = sentiment_pipe(df.text2.tolist()[100*i:np.min([100*i+100,len(df)])])
    text1_sent+=t1_s
    text2_sent+=t2_s
    pbar.update()
pbar.close()
len(text1_sent)

In [ ]:
sent = {'POSITIVE':1,'NEGATIVE':-1}

df['sentiment_1'] = np.array([x['score']*sent[x['label']] for x in text1_sent]) 
df['sentiment_2'] = np.array([x['score']*sent[x['label']] for x in text2_sent])
df['dif_sent'] =  np.abs(df['sentiment_1']-df['sentiment_2'])

In [ ]:
pairs = []
for index, row in df.iterrows():
    first_sentence_tokens = row['text1'].strip().split()
    second_sentence_tokens = row['text2'].strip().split()
    pairs.append((first_sentence_tokens, second_sentence_tokens))

In [ ]:
scores_bleu1 = []
for first_sentence_tokens, second_sentence_tokens in pairs:

    score_bleu1 = sentence_bleu([first_sentence_tokens], second_sentence_tokens, weights=(1, 0, 0, 0))
    scores_bleu1.append(score_bleu1)

print(np.mean(scores_bleu1))
print(np.std(scores_bleu1))


In [ ]:
df['bleu_score_1'] = scores_bleu1

In [ ]:
annot_std_semantic = df[(df['bleu_score_1'] > 0.8) & (df['dif_sent'] > 1.9)].groupby('annotator')['label'].std().dropna()
ba_semantics = list(annot_std_semantic[annot_std_semantic > 1.0].index)

# Combining all the results

In [ ]:
import itertools

In [ ]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

In [ ]:
all_ba = ['ba_semantics','ba_time','ba_unpopular','ba_unvar_annotations']

for a,b in list(itertools.combinations(all_ba,2)):
    print(f"Jaccard Similarity of {a} and {b} is :{jaccard_similarity(eval(a),eval(b))}")

While the overlap isn't super consistent, it is interesting to note that the two most correlated groups are time and unpopularity and unpopularity with unvaried annotations.

In [ ]:
all_ba = list(set(ba_unvar_annotations + ba_unpopular + ba_time + ba_semantics))
print(f"Total number of bad annotators are: {len(all_ba)}")
print(f"Percentage of total annotators are: {len(all_ba)/df.annotator.nunique()}")

### Save the annotators so we can filter them out quicker later

In [ ]:
for ba in all_ba:
    with open(f'data/other/{ba}.txt', 'w') as f:
        for item in eval(ba):
            f.write("%s\n" % item)

In [ ]:
with open('data/other/ba_all.txt','w') as f:
    for item in list(set(ba_unvar_annotations + ba_unpopular + ba_time + ba_semantics)):
        f.write("%s\n" % item)